In [1]:
%cd ..

/home/melek/workspace/vscode/nb_gpu_trainer


In [2]:
!pip install -e .

Obtaining file:///home/melek/workspace/vscode/nb_gpu_trainer
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for nb_gpu_trainer (pyproject.toml) ... done
  Created wheel for nb_gpu_trainer: filename=nb_gpu_trainer-1.0-0.editable-py3-none-any.whl size=2181 sha256=b9bf2113764975df2d9e62a6f5d6edb3ba00e02cfaa111d630cda79ef1a95fb1
  Stored in directory: /tmp/pip-ephem-wheel-cache-bah8qky7/wheels/79/92/70/55487793b668bb63a254086efaa207c1fa25d66f02098d5bd3
Successfully built nb_gpu_trainer
  Attempting uninstall: nb_gpu_trainer
    Found existing installation: nb_gpu_trainer 1.0
    Uninstalling nb_gpu_trainer-1.0:
      Successfully uninstalled nb_gpu_trainer-1.0


# Train a tokenizer

In [3]:
from tokenizer.train import Trainer
from huggingface_hub import login
from trainer import TrainerConfig

In [4]:
login()

In [5]:
def load_config(config_file)->TrainerConfig:
    import yaml

    with open(config_file) as f:
        try:
            doc = yaml.safe_load(f)
        except yaml.YAMLError as exc:
            print(exc)
            exit(1)
    
    return TrainerConfig(**doc)

trainer_cfg = load_config("config/news_trainer.yml")

In [6]:
trainer_cfg.repo_id, trainer_cfg.out_dir, trainer_cfg.ds_repo_id

('habanoz/haber-90k-gpt-v1.0', 'haber-90k-gpt', 'habanoz/haber-90k-gpt-text')

## Train and push

In [7]:
from datasets import load_dataset, DatasetDict

In [8]:
ds = load_dataset("habanoz/news-tr-90k")

Generating train split:   0%|          | 0/81708 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/9079 [00:00<?, ? examples/s]

In [9]:
dss = ds['train'].train_test_split(0.1)

In [10]:
dss

DatasetDict({
    train: Dataset({
        features: ['Id', 'Tag', 'Labels', 'Title', 'Summary', 'Text', '__index_level_0__'],
        num_rows: 81708
    })
    test: Dataset({
        features: ['Id', 'Tag', 'Labels', 'Title', 'Summary', 'Text', '__index_level_0__'],
        num_rows: 9079
    })
})

In [11]:
def to_doc(x):
    return f"""# {x['Title']}

## Özet

{x['Summary']}

## İçerik

{x['Text']}"""

In [12]:
dss = dss.map(lambda x: {'doc':to_doc(x)}, batched=False)

Map:   0%|          | 0/81708 [00:00<?, ? examples/s]

Map:   0%|          | 0/9079 [00:00<?, ? examples/s]

In [13]:
dss

DatasetDict({
    train: Dataset({
        features: ['Id', 'Tag', 'Labels', 'Title', 'Summary', 'Text', '__index_level_0__', 'doc'],
        num_rows: 81708
    })
    test: Dataset({
        features: ['Id', 'Tag', 'Labels', 'Title', 'Summary', 'Text', '__index_level_0__', 'doc'],
        num_rows: 9079
    })
})

In [14]:
dss = DatasetDict({
    "train":dss['train'],
    "validation":dss['test'],
})

In [15]:
dss

DatasetDict({
    train: Dataset({
        features: ['Id', 'Tag', 'Labels', 'Title', 'Summary', 'Text', '__index_level_0__', 'doc'],
        num_rows: 81708
    })
    validation: Dataset({
        features: ['Id', 'Tag', 'Labels', 'Title', 'Summary', 'Text', '__index_level_0__', 'doc'],
        num_rows: 9079
    })
})

In [16]:
dss = dss.remove_columns(['Id', 'Tag', 'Labels', 'Title', 'Summary', 'Text', '__index_level_0__'])

In [18]:
dss = dss.rename_column('doc','text')

In [20]:
dss.push_to_hub("habanoz/haber-90k-gpt-text")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/82 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/habanoz/haber-90k-gpt-text/commit/c02f66b5bdb10cac0c062492f39f6a6adc8ec3a6', commit_message='Upload dataset', commit_description='', oid='c02f66b5bdb10cac0c062492f39f6a6adc8ec3a6', pr_url=None, pr_revision=None, pr_num=None)